# Waste Sorting - YOLOv8 Object Detection
### Detect and classify multiple waste objects using YOLOv8

In [1]:
import os
from ultralytics import YOLO
from pathlib import Path
import shutil


In [2]:
# Define classes
classes = ['cardboard','glass','metal','paper','plastic','trash']

# Function to generate YOLO labels for single-object-per-image
def create_yolo_labels(img_folder, label_folder):
    os.makedirs(label_folder, exist_ok=True)
    for class_idx, cls in enumerate(classes):
        cls_folder = os.path.join(img_folder, cls)
        for img_name in os.listdir(cls_folder):
            if img_name.lower().endswith(('.png','.jpg','.jpeg')):
                txt_path = os.path.join(label_folder, img_name.replace('.jpg','.txt').replace('.png','.txt'))
                # Full image box: x_center=0.5, y_center=0.5, width=1, height=1
                with open(txt_path, 'w') as f:
                    f.write(f"{class_idx} 0.5 0.5 1.0 1.0\n")

# Example: create labels for training set
create_yolo_labels('data/train', 'data/train_labels')
create_yolo_labels('data/val', 'data/val_labels')


In [3]:
data_yaml = """
train: data/train
val: data/val
test: data/test

nc: 6
names: ['cardboard','glass','metal','paper','plastic','trash']
"""

with open("waste_dataset.yaml", "w") as f:
    f.write(data_yaml)


In [4]:
import torch

if torch.cuda.is_available():
    print("✅ CUDA is available! GPU detected.")
    print(f"Number of GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory Allocated: {torch.cuda.memory_allocated(i)/1024**2:.2f} MB")
        print(f"  Memory Cached:    {torch.cuda.memory_reserved(i)/1024**2:.2f} MB")
else:
    print("❌ No GPU detected. Using CPU.")

✅ CUDA is available! GPU detected.
Number of GPUs: 1
GPU 0: NVIDIA GeForce RTX 3060
  Memory Allocated: 0.00 MB
  Memory Cached:    0.00 MB


In [ ]:
model = YOLO("yolov8n.pt")  
model.to("cuda") 

# Train model
model.train(
    data="waste_dataset.yaml",
    epochs=20,
    imgsz=224,
    batch=16,
    name="yolov8_waste",
    device=0
)

Ultralytics 8.3.191  Python-3.11.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=waste_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_waste3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, p

In [ ]:
# Evaluate on validation set
metrics = model.val()
print(metrics)


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load a sample test image
test_img = "data/test/plastic/sample1.jpg"

results = model.predict(source=test_img, save=True)  # saves result in runs/predict/

# Show image using matplotlib
img = cv2.imread(f"runs/predict/{Path(test_img).name}")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.axis('off')
plt.show()


In [1]:
import torch

if torch.cuda.is_available():
    print("✅ CUDA available! GPU detected.")
    print("Number of GPUs:", torch.cuda.device_count())
    print("GPU 0:", torch.cuda.get_device_name(0))
else:
    print("❌ No GPU detected. YOLOv8 will run on CPU (slow).")


✅ CUDA available! GPU detected.
Number of GPUs: 1
GPU 0: NVIDIA GeForce RTX 3060


In [2]:
import os

classes = ['cardboard','glass','metal','paper','plastic','trash']

def create_yolo_labels(img_folder, label_folder):
    os.makedirs(label_folder, exist_ok=True)
    for class_idx, cls in enumerate(classes):
        cls_folder = os.path.join(img_folder, cls)
        for img_name in os.listdir(cls_folder):
            if img_name.lower().endswith(('.jpg','.png','.jpeg')):
                txt_path = os.path.join(label_folder, img_name.rsplit('.',1)[0]+'.txt')
                with open(txt_path, 'w') as f:
                    f.write(f"{class_idx} 0.5 0.5 1.0 1.0\n")

# Generate labels for training and validation
create_yolo_labels('data/train', 'data/train_labels')
create_yolo_labels('data/val', 'data/val_labels')


In [3]:
data_yaml = """
train: data/train
val: data/val
test: data/test

nc: 6
names: ['cardboard','glass','metal','paper','plastic','trash']
"""

with open("waste_dataset.yaml", "w") as f:
    f.write(data_yaml)


In [4]:
from ultralytics import YOLO

# Use YOLOv8n (nano) pretrained on COCO for transfer learning
model = YOLO("yolov8n.pt")


In [ ]:
model.train(
    data="waste_dataset.yaml",  # dataset info
    epochs=20,                  # fine-tune for 20 epochs
    imgsz=64,                   # image size
    batch=16,                   # batch size
    name="yolov8_waste",        # folder to save results
    device=0,                   # use GPU 0
    cache=True 
)


Ultralytics 8.3.191  Python-3.11.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=waste_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_waste6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pre

In [ ]:
metrics = model.val()
print(metrics)  # prints mAP, precision, recall


In [ ]:
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

test_img = "data/test/plastic/sample1.jpg"

# Run prediction (GPU will be used)
results = model.predict(source=test_img, save=True)

# Display saved result
img = cv2.imread(f"runs/predict/{Path(test_img).name}")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.axis('off')
plt.show()


In [ ]:
# Notebook: YOLOv8 Waste Detection (Quick Test with 50 Images/Class)
# ================================================================

# Markdown
"""
# YOLOv8 Waste Detection - Quick Training
This notebook creates a small dataset (50 images per class) flattened into a single folder, 
generates YOLO labels, and fine-tunes YOLOv8 on GPU.
"""

# 1️⃣ Imports
import os
import shutil
import random
from pathlib import Path
from ultralytics import YOLO
import torch

# 2️⃣ Configuration
SRC_DATA_DIR = Path("data/train")      # Original dataset
DST_DATA_DIR = Path("yolo_subset")     # Flattened dataset
CLASSES = ["cardboard","glass","metal","paper","plastic","trash"]
IMAGES_PER_CLASS = 50
IMG_EXT = [".jpg", ".png", ".jpeg"]

TRAIN_DIR = DST_DATA_DIR / "images" / "train"
VAL_DIR = DST_DATA_DIR / "images" / "val"
LABEL_DIR_TRAIN = DST_DATA_DIR / "labels" / "train"
LABEL_DIR_VAL = DST_DATA_DIR / "labels" / "val"

# 3️⃣ Create necessary folders
for folder in [TRAIN_DIR, VAL_DIR, LABEL_DIR_TRAIN, LABEL_DIR_VAL]:
    os.makedirs(folder, exist_ok=True)

# 4️⃣ Copy 50 images per class and split 80/20 train/val
for cls_idx, cls_name in enumerate(CLASSES):
    cls_path = SRC_DATA_DIR / cls_name
    imgs = [f for f in cls_path.iterdir() if f.suffix.lower() in IMG_EXT]
    sampled = random.sample(imgs, min(IMAGES_PER_CLASS, len(imgs)))
    
    # Split 80% train, 20% val
    n_train = int(0.8 * len(sampled))
    train_imgs = sampled[:n_train]
    val_imgs = sampled[n_train:]
    
    # Copy to destination folder
    for img in train_imgs:
        dst_img = TRAIN_DIR / f"{cls_name}_{img.name}"
        shutil.copy(img, dst_img)
        # Create YOLO label for single-object full-image
        label_path = LABEL_DIR_TRAIN / f"{cls_name}_{img.stem}.txt"
        with open(label_path, "w") as f:
            f.write(f"{cls_idx} 0.5 0.5 1 1\n")  # single object filling image
    
    for img in val_imgs:
        dst_img = VAL_DIR / f"{cls_name}_{img.name}"
        shutil.copy(img, dst_img)
        label_path = LABEL_DIR_VAL / f"{cls_name}_{img.stem}.txt"
        with open(label_path, "w") as f:
            f.write(f"{cls_idx} 0.5 0.5 1 1\n")

print("✅ Subset dataset created with 50 images per class.")

# 5️⃣ Create YOLOv8 dataset YAML
yaml_content = f"""
train: {TRAIN_DIR.resolve()}
val: {VAL_DIR.resolve()}

nc: {len(CLASSES)}
names: {CLASSES}
"""

yaml_file = DST_DATA_DIR / "waste_subset.yaml"
with open(yaml_file, "w") as f:
    f.write(yaml_content)

print(f"✅ YAML file saved at {yaml_file}")

# 6️⃣ Check GPU
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

# 7️⃣ Load YOLOv8 model
model = YOLO("yolov8n.pt")  # smallest YOLOv8 model, fast for quick tests

# 8️⃣ Train model
model.train(
    data=str(yaml_file),
    epochs=5,
    imgsz=64,
    batch=8,
    name="yolov8_waste_quick",
    device=0,   # GPU 0
    cache=True, # speed up dataset loading
    verbose=True
)


✅ Subset dataset created with 50 images per class.
✅ YAML file saved at yolo_subset\waste_subset.yaml
CUDA available: True
GPU name: NVIDIA GeForce RTX 3060
Ultralytics 8.3.191  Python-3.11.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_subset\waste_subset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=

In [ ]:
# 9️⃣ Evaluate the trained model

# Run validation on the same validation set
results = model.val()  # YOLOv8 automatically uses the val set from YAML

# results contains metrics: mAP50, mAP50-95, precision, recall, loss
print("✅ Validation Metrics:")
print(f"mAP50: {results.metrics['mAP_0.5']:.3f}")
print(f"mAP50-95: {results.metrics['mAP_0.5:0.95']:.3f}")
print(f"Precision: {results.metrics['precision']:.3f}")
print(f"Recall: {results.metrics['recall']:.3f}")

# 10️⃣ Generate predictions on validation set for confusion matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

y_true = []
y_pred = []

# Loop over validation images
val_images = list(VAL_DIR.glob("*.*"))
for img_path in val_images:
    img_class_name = img_path.name.split("_")[0]  # original class
    y_true.append(CLASSES.index(img_class_name))
    
    # Predict
    pred = model.predict(str(img_path), verbose=False)[0]
    
    if len(pred.boxes) > 0:
        # Take highest confidence prediction
        confs = pred.boxes.conf.cpu().numpy()
        labels = pred.boxes.cls.cpu().numpy()
        best_idx = np.argmax(confs)
        y_pred.append(int(labels[best_idx]))
    else:
        # No detection → assign -1 (can adjust to background class if needed)
        y_pred.append(-1)

# Remove predictions with -1 if you want only correct detections
mask = [p >= 0 for p in y_pred]
y_true_filtered = np.array(y_true)[mask]
y_pred_filtered = np.array(y_pred)[mask]

# Confusion matrix
cm = confusion_matrix(y_true_filtered, y_pred_filtered)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=CLASSES, yticklabels=CLASSES, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("YOLOv8 Confusion Matrix")
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(y_true_filtered, y_pred_filtered, target_names=CLASSES))
